In [1]:
from __future__ import print_function
import numpy as np
from numpy import newaxis as na
import keras

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM
from keras.datasets import imdb
from keras.models import Model
from keras import backend as K

import matplotlib.pyplot as plt
from IPython.display import display, HTML


Using TensorFlow backend.


In [2]:
max_features = 10000

maxlen = 20
batch_size = 32

print('Loading data...')

np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
np.load = np_load_old  

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [3]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
# One-hot encoding the output
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=[x_test, y_test])

Pad sequences (samples x time)


W0811 17:49:00.587483 140043398403968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0811 17:49:00.603104 140043398403968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0811 17:49:00.606318 140043398403968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



x_train shape: (25000, 20)
x_test shape: (25000, 20)


W0811 17:49:00.845208 140043398403968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0811 17:49:00.857350 140043398403968 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0811 17:49:00.889700 140043398403968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0811 17:49:00.914364 140043398403968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tenso

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 35s 1ms/step - loss: 0.5274 - acc: 0.7271 - val_loss: 0.4792 - val_acc: 0.7667
Epoch 2/10
25000/25000 [==============================] - 34s 1ms/step - loss: 0.3942 - acc: 0.8186 - val_loss: 0.4945 - val_acc: 0.7629
Epoch 3/10
25000/25000 [==============================] - 34s 1ms/step - loss: 0.3108 - acc: 0.8629 - val_loss: 0.5339 - val_acc: 0.7580
Epoch 4/10
25000/25000 [==============================] - 34s 1ms/step - loss: 0.2375 - acc: 0.8991 - val_loss: 0.6309 - val_acc: 0.7514
Epoch 5/10
25000/25000 [==============================] - 34s 1ms/step - loss: 0.1706 - acc: 0.9313 - val_loss: 0.7817 - val_acc: 0.7409
Epoch 6/10
25000/25000 [==============================] - 34s 1ms/step - loss: 0.1241 - acc: 0.9530 - val_loss: 1.0387 - val_acc: 0.7315
Epoch 7/10
25000/25000 [==============================] - 34s 1ms/step - loss: 0.0877 - acc: 0.9685 - val_loss: 1.1137 

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 1,329,538
Trainable params: 1,329,538
Non-trainable params: 0
_________________________________________________________________


In [5]:
results = model.evaluate(x_test, y_test)

25000/25000 [==============================] - 4s 142us/step


In [6]:
results

[1.557992409286499, 0.72668]

In [0]:
def get_layer_output(layer_name, data):
    # https://keras.io/getting-started/faq/#how-can-i-obtain-the-output-of-an-intermediate-layer
    intermediate_layer_model = keras.Model(inputs=model.input,
                                     outputs=model.get_layer(layer_name).output)
    return intermediate_layer_model.predict(data)  

In [0]:
names = [weight.name for layer in model.layers for weight in layer.weights]
weights = model.get_weights()

# suppress scientific notation
np.set_printoptions(suppress=True)
for name, weight in zip(names, weights):
    if name == 'lstm_1/kernel:0':
        kernel_0 = weight
    if name == 'lstm_1/recurrent_kernel:0':
        recurrent_kernel_0 = weight
    if name == 'lstm_1/bias:0':
        bias_0 = weight
    elif name == 'dense_1/kernel:0':
        output = weight

In [9]:
print("kernel_0", kernel_0.shape)
print("recurrent_kernel_0", recurrent_kernel_0.shape)
print("bias_0", bias_0.shape)
print("output", output.shape)

# self.Wxh_Left (240, 60)
# self.Whh_Left (240, 60)
# self.bxh_Left (240,)
# self.Why_Left (5, 60)

kernel_0 (128, 256)
recurrent_kernel_0 (64, 256)
bias_0 (256,)
output (64, 2)


In [10]:
Wxh = kernel_0.T  # shape 4d*e
Whh = recurrent_kernel_0.T  # shape 4d
bxh = bias_0.T  # shape 4d 
Why = output.T

print("Wxh", Wxh.shape)
print("Whh", Whh.shape)
print("bxh", bxh.shape)
print("Why", Why.shape)

Wxh (256, 128)
Whh (256, 64)
bxh (256,)
Why (2, 64)


In [0]:
def html_heatmap (words, scores, cmap_name="bwr"):
    """
    Return word-level heatmap in HTML format,
    with words being the list of words (as string),
    scores the corresponding list of word-level relevance values,
    and cmap_name the name of the matplotlib diverging colormap.
    """
    
    colormap  = plt.get_cmap(cmap_name)
     
    #assert len(words)==len(scores)
    max_s     = max(scores)
    min_s     = min(scores)
    
    output_text = ""
    
    for idx, w in enumerate(words):
        score       = rescale_score_by_abs(scores[idx], max_s, min_s)
        output_text = output_text + span_word(w, score, colormap) + " "
    
    return output_text + "\n"

def rescale_score_by_abs (score, max_score, min_score):
    """
    Normalize the relevance value (=score), accordingly to the extremal relevance values (max_score and min_score), 
    for visualization with a diverging colormap.
    i.e. rescale positive relevance to the range [0.5, 1.0], and negative relevance to the range [0.0, 0.5],
    using the highest absolute relevance for linear interpolation.
    """
    
    # CASE 1: positive AND negative scores occur --------------------
    if max_score>0 and min_score<0:
    
        if max_score >= abs(min_score):   # deepest color is positive
            if score>=0:
                return 0.5 + 0.5*(score/max_score)
            else:
                return 0.5 - 0.5*(abs(score)/max_score)

        else:                             # deepest color is negative
            if score>=0:
                return 0.5 + 0.5*(score/abs(min_score))
            else:
                return 0.5 - 0.5*(score/min_score)   
    
    # CASE 2: ONLY positive scores occur -----------------------------       
    elif max_score>0 and min_score>=0: 
        if max_score == min_score:
            return 1.0
        else:
            return 0.5 + 0.5*(score/max_score)
    
    # CASE 3: ONLY negative scores occur -----------------------------
    elif max_score<=0 and min_score<0: 
        if max_score == min_score:
            return 0.0
        else:
            return 0.5 - 0.5*(score/min_score)
          

def getRGB (c_tuple):
    return "#%02x%02x%02x"%(int(c_tuple[0]*255), int(c_tuple[1]*255), int(c_tuple[2]*255))

     
def span_word (word, score, colormap):
    return "<span style=\"background-color:"+getRGB(colormap(score))+"\">"+word+"</span>"

In [0]:
def lrp_linear(hin, w, b, hout, Rout, bias_nb_units, eps, bias_factor=0.0, debug=False):
    """
    LRP for a linear layer with input dim D and output dim M.
    Args:
    - hin:            forward pass input, of shape (D,)
    - w:              connection weights, of shape (D, M)
    - b:              biases, of shape (M,)
    - hout:           forward pass output, of shape (M,) (unequal to np.dot(w.T,hin)+b if more than one incoming layer!)
    - Rout:           relevance at layer output, of shape (M,)
    - bias_nb_units:  total number of connected lower-layer units (onto which the bias/stabilizer contribution is redistributed for sanity check)
    - eps:            stabilizer (small positive number)
    - bias_factor:    set to 1.0 to check global relevance conservation, otherwise use 0.0 to ignore bias/stabilizer redistribution (recommended)
    Returns:
    - Rin:            relevance at layer input, of shape (D,)
    """
    sign_out = np.where(hout[na,:]>=0, 1., -1.) # shape (1, M)
    
    numer    = (w * hin[:,na]) + ( bias_factor * (b[na,:]*1. + eps*sign_out*1.) / bias_nb_units ) # shape (D, M)
    # Note: here we multiply the bias_factor with both the bias b and the stabilizer eps since in fact
    # using the term (b[na,:]*1. + eps*sign_out*1.) / bias_nb_units in the numerator is only useful for sanity check
    # (in the initial paper version we were using (bias_factor*b[na,:]*1. + eps*sign_out*1.) / bias_nb_units instead)
    
    denom    = hout[na,:] + (eps*sign_out*1.)   # shape (1, M)
    
    message  = (numer/denom) * Rout[na,:]       # shape (D, M)
    
    Rin      = message.sum(axis=1)              # shape (D,)
    
    if debug:
        print("local diff: ", Rout.sum() - Rin.sum())
    # Note: 
    # - local  layer   relevance conservation if bias_factor==1.0 and bias_nb_units==D (i.e. when only one incoming layer)
    # - global network relevance conservation if bias_factor==1.0 and bias_nb_units set accordingly to the total number of lower-layer connections 
    # -> can be used for sanity check
    
    return Rin

In [0]:
def LRP(target_data, target_class) :
    
    #원본 소스에서 E embedding은 전체에 대한 단어 사전이고, x는 embedding된 인풋이다.  
    # w_indices [109, 11995, 25, 18263, 25, 973, 3138, 6389, 372]

    x = get_layer_output('embedding_1', target_data).squeeze(axis=1)
    e = x.shape[1]

   ################# forword
    T = target_data.shape[0]
    d = int(256/4)  # hidden units
    C = Why.shape[0] # number of classes
    
    idx    = np.hstack((np.arange(0,d), np.arange(2*d,4*d))).astype(int) # indices of gates i,f,o together
    idx_i, idx_g, idx_f, idx_o = np.arange(0,d), np.arange(d,2*d), np.arange(2*d,3*d), np.arange(3*d,4*d) # indices of gates i,g,f,o separately

    h  = np.zeros((T,d))
    c  = np.zeros((T,d))

    gates_xh  = np.zeros((T, 4*d))  
    gates_hh  = np.zeros((T, 4*d)) 
    gates_pre = np.zeros((T, 4*d))  
    gates     = np.zeros((T, 4*d))  

    for t in range(T):
        gates_xh[t]     = np.dot(Wxh, x[t])
        gates_hh[t]     = np.dot(Whh, h[t-1])
        gates_pre[t]    = gates_xh[t] + gates_hh[t] + bxh
        gates[t, idx]    = 1.0/(1.0 + np.exp(- gates_pre[t,idx]))
        gates[t,idx_g]  = np.tanh(gates_pre[t,idx_g]) 
        c[t]            = gates[t,idx_f]*c[t-1] + gates[t,idx_i]*gates[t,idx_g]
        h[t]            = gates[t,idx_o]*np.tanh(c[t])

    s = np.dot(Why, h[t])    

    ################# backwork
    dx     = np.zeros(x.shape)

    dh          = np.zeros((T, d))
    dc          = np.zeros((T, d))
    dgates_pre  = np.zeros((T, 4*d))  # gates pre-activation
    dgates      = np.zeros((T, 4*d))  # gates activation

    ds               = np.zeros((C))
    ds[target_class] = 1.0
    dy               = ds.copy()

    #맨처음을 0으로 시작하지 않게 위한조치
    dh[T-1]     = np.dot(Why.T, dy)

    for t in reversed(range(T)): 
        dgates[t,idx_o]    = dh[t] * np.tanh(c[t])  # do[t]
        dc[t]             += dh[t] * gates[t,idx_o] * (1.-(np.tanh(c[t]))**2) # dc[t]
        dgates[t,idx_f]    = dc[t] * c[t-1]         # df[t]
        dc[t-1]            = dc[t] * gates[t,idx_f] # dc[t-1]
        dgates[t,idx_i]    = dc[t] * gates[t,idx_g] # di[t]
        dgates[t,idx_g]    = dc[t] * gates[t,idx_i] # dg[t]
        dgates_pre[t,idx]  = dgates[t,idx] * gates[t,idx] * (1.0 - gates[t,idx]) # d ifo pre[t]
        dgates_pre[t,idx_g]= dgates[t,idx_g] *  (1.-(gates[t,idx_g])**2) # d g pre[t]
        dh[t-1]            = np.dot(Whh.T, dgates_pre[t])
        dx[t]              = np.dot(Wxh.T, dgates_pre[t])

    ################# LRP
    eps=0.001 
    bias_factor=0.0
    Rx  = np.zeros(x.shape)
    Rh  = np.zeros((T+1, d))
    Rc  = np.zeros((T+1, d))
    Rg  = np.zeros((T,   d)) # gate g only

    Rout_mask            = np.zeros((C))
    Rout_mask[target_class] = 1.0  

    # format reminder: lrp_linear(hin, w, b, hout, Rout, bias_nb_units, eps, bias_factor)
    Rh[T-1]  = lrp_linear(h[T-1], Why.T, np.zeros((C)), s, s*Rout_mask, 2*d, eps, bias_factor, debug=False)  

    for t in reversed(range(T)):
        Rc[t]   += Rh[t]
        Rc[t-1]  = lrp_linear(gates[t,idx_f]*c[t-1], np.identity(d), np.zeros((d)), c[t], Rc[t], 2*d, eps, bias_factor, debug=False)
        Rg[t]    = lrp_linear(gates[t,idx_i]*gates[t,idx_g], np.identity(d), np.zeros((d)), c[t], Rc[t], 2*d, eps, bias_factor, debug=False)
        Rx[t]    = lrp_linear(x[t], Wxh[idx_g].T, bxh[idx_g], gates_pre[t,idx_g], Rg[t], d+e, eps, bias_factor, debug=False)
        Rh[t-1]  = lrp_linear(h[t-1], Whh[idx_g].T, bxh[idx_g], gates_pre[t,idx_g], Rg[t], d+e, eps, bias_factor, debug=False)    

    return s, dx, Rx, Rh[-1].sum()

In [0]:
word_index = imdb.get_word_index()
index_word = {v:k for k,v in word_index.items()}

def index_to_word(word):
    full_sentence = ' '.join(index_word.get(w) for w in word)
    return full_sentence.split()

In [0]:
def int_to_str(target_class):
    if target_class == 0 :
        return "부정"
    else :
        return "긍정"

In [0]:
predictions = model.predict(x_test)

In [0]:
positive_list = []
negative_list = []

for i in range(len(y_test)):
    if np.argmax(predictions[i]) == 1:
        positive_list.append(i)
    else :
        negative_list.append(i)

In [18]:
y_test[0]

array([1., 0.], dtype=float32)

In [19]:
print("각 단어가 긍정이라고 예측하는데 영향을 미치는 정도" )
for index, i in enumerate(positive_list):
    
    target_data = x_test[i]
    target_class = np.argmax(y_test[i])

    scores, Gx, Rx, R_rest = LRP(target_data, target_class)

    R_words          = np.sum(Rx, axis=1)                       # compute word-level LRP relevances
    R_words_SA       = (np.linalg.norm(Gx,ord=2, axis=0))**2   # compute word-level Sensitivity Analysis relevances
    R_words_GI       = np.dot(target_data, Gx) 

    words = index_to_word(target_data)

    if len(words) > 0 :
        print(" 예측 레이블:", int_to_str(np.argmax(predictions[i])), "| 실제 레이블 : ", int_to_str(target_class))

        print("        LRP heatmap:")
        display(HTML(html_heatmap(words, R_words)))

        print("        SA heatmap:")
        display(HTML(html_heatmap(words, R_words_SA)))

        print("        GI heatmap:")
        display(HTML(html_heatmap(words, R_words_GI)))

        print("-----------------------------------------------------------")

    if index == 100: # 50개만 출력
      break

각 단어가 긍정이라고 예측하는데 영향을 미치는 정도
 예측 레이블: 긍정 | 실제 레이블 :  긍정
        LRP heatmap:


        SA heatmap:


        GI heatmap:


-----------------------------------------------------------
 예측 레이블: 긍정 | 실제 레이블 :  긍정
        LRP heatmap:


        SA heatmap:


...

        GI heatmap:


-----------------------------------------------------------
 예측 레이블: 긍정 | 실제 레이블 :  긍정
        LRP heatmap:


        SA heatmap:


        GI heatmap:


-----------------------------------------------------------


In [20]:
output = get_layer_output('dense_1', x)

NameError: ignored

In [0]:
word_index = imdb.get_word_index()
index_word = {v:k for k,v in word_index.items()}
' '.join(index_word.get(w) for w in x)

In [0]:
full_sentence = ' '.join(index_word.get(w) for w in x_train[9])
words = full_sentence.split()

In [0]:
words = full_sentence.split()

In [0]:
n = 100
T = 128
d = int(256/4)  # LSTM layers

def hard_sigmoid(x):
    return np.clip(0.2 * x + 0.5, 0, 1)

Wi = kernel_0[:, 0:d]
Wf = kernel_0[:, d:2 * d]
Wc = kernel_0[:, 2 * d:3 * d]
Wo = kernel_0[:, 3 * d:]


Ui = recurrent_kernel_0[:, 0:d]
Uf = recurrent_kernel_0[:, d:2 * d]
Uc = recurrent_kernel_0[:, 2 * d:3 * d]
Uo = recurrent_kernel_0[:, 3 * d:]

bi = bias_0[0:d]
bf = bias_0[d:2 * d]
bc = bias_0[2 * d:3 * d]
bo = bias_0[3 * d:]


Ct = np.zeros((T, d))
ht = np.zeros((T, d)) 


for t in range(n):
    xt = np.array(e[t, :])

    ft = hard_sigmoid(np.dot(xt, Wf) + np.dot(ht[t], Uf) + bf)  # forget gate
    it = hard_sigmoid(np.dot(xt, Wi) + np.dot(ht[t], Ui) + bi)  # input gate
    ot = hard_sigmoid(np.dot(xt, Wo) + np.dot(ht[t], Uo) + bo)  # output gate
    

    Ct = ft * Ct + it * np.tanh(np.dot(xt, Wc) + np.dot(ht, Uc) + bc)
    ht = ot * np.tanh(Ct)
    
s = np.dot(ht[0], output_0)

In [0]:
print(s)
print(true_y)

In [0]:
#LSTM코드방식 
n = 100
T = 128
d = int(256/4)  # LSTM layers


Ct = np.zeros((T, d))
ht = np.zeros((T, d)) 

idx    = np.hstack((np.arange(0,d), np.arange(2*d,4*d))).astype(int) # indices of gates i,f,o together
idx_i, idx_g, idx_f, idx_o = np.arange(0,d), np.arange(d,2*d), np.arange(2*d,3*d), np.arange(3*d,4*d) # indices of gates i,g,f,o separately

h_Left         = np.zeros((T+1, d))
c_Left         = np.zeros((T+1, d))

gates_xh_Left  = np.zeros((T, 4*d))  
gates_hh_Left  = np.zeros((T, 4*d)) 
gates_pre_Left = np.zeros((T, 4*d))  # gates pre-activation
gates_Left     = np.zeros((T, 4*d)) 


for t in range(n):
    gates_xh_Left[t]     = np.dot(e[t], kernel_0)
    gates_hh_Left[t]     = np.dot(h_Left[t-1],recurrent_kernel_0) 
    gates_pre_Left[t]    = gates_xh_Left[t] + gates_hh_Left[t] + bias_0
    gates_Left[t,idx]    = 1.0/(1.0 + np.exp(- gates_pre_Left[t,idx]))
    gates_Left[t,idx_g]  = np.tanh(gates_pre_Left[t,idx_g]) 
    c_Left[t]            = gates_Left[t,idx_f]*c_Left[t-1] + gates_Left[t,idx_i]*gates_Left[t,idx_g]
    h_Left[t]            = gates_Left[t,idx_o]*np.tanh(c_Left[t])

score = np.dot(h_Left[t], output_0)

In [0]:
score

In [0]:
output_0.shape

In [0]:
sensitivity_class = 1

In [0]:
c_Left[99]

In [0]:
dh_Left[T-1]

In [0]:
#backprop

C      = output_0.shape[1]   # number of classes
dx     = np.zeros(e.shape)


dh_Left          = np.zeros((T+1, d))
dc_Left          = np.zeros((T+1, d))
dgates_pre_Left  = np.zeros((T, 4*d))  # gates pre-activation
dgates_Left      = np.zeros((T, 4*d))  # gates activation

ds                    = np.zeros((C))
ds[sensitivity_class] = 1.0
dy_Left               = ds.copy()

#맨처음을 0으로 시작하지 않게 위한조치
dh_Left[n-1]     = np.dot(dy_Left, output_0.T)

for t in reversed(range(n)): 
    dgates_Left[t,idx_o]    = dh_Left[t] * np.tanh(c_Left[t])  # do[t]
    dc_Left[t]             += dh_Left[t] * gates_Left[t,idx_o] * (1.-(np.tanh(c_Left[t]))**2) # dc[t]
    dgates_Left[t,idx_f]    = dc_Left[t] * c_Left[t-1]         # df[t]
    dc_Left[t-1]            = dc_Left[t] * gates_Left[t,idx_f] # dc[t-1]
    dgates_Left[t,idx_i]    = dc_Left[t] * gates_Left[t,idx_g] # di[t]
    dgates_Left[t,idx_g]    = dc_Left[t] * gates_Left[t,idx_i] # dg[t]
    dgates_pre_Left[t,idx]  = dgates_Left[t,idx] * gates_Left[t,idx] * (1.0 - gates_Left[t,idx]) # d ifo pre[t]
    dgates_pre_Left[t,idx_g]= dgates_Left[t,idx_g] *  (1.-(gates_Left[t,idx_g])**2) # d g pre[t]
    dh_Left[t-1]            = np.dot(dgates_pre_Left[t], recurrent_kernel_0.T)
    dx[t]                   = np.dot(dgates_pre_Left[t], kernel_0.T)

In [0]:
Gx.shape

In [0]:
x.shape

In [0]:
Gx.sum(axis=0) * x

In [0]:
target_class = 1
w_indices = x
Gx       = np.squeeze(e.squeeze().T)             # perform gradient backpropagation
R_words_SA       = (np.linalg.norm(Gx, ord=2, axis=1))**2   # compute word-level Sensitivity Analysis relevances
R_words_GI       = np.dot(x, Gx)                 # compute word-level GradientxInput relevances

In [0]:
np.dot(Gx, e.squeeze().T).sum(axis=1)

In [0]:
test = Gx * x

In [0]:
R_words_GI

In [0]:
import matplotlib.pyplot as plt

In [0]:

def html_heatmap (words, scores, cmap_name="bwr"):
    """
    Return word-level heatmap in HTML format,
    with words being the list of words (as string),
    scores the corresponding list of word-level relevance values,
    and cmap_name the name of the matplotlib diverging colormap.
    """
    
    colormap  = plt.get_cmap(cmap_name)
     
    assert len(words)==len(scores)
    max_s     = max(scores)
    min_s     = min(scores)
    
    output_text = ""
    
    for idx, w in enumerate(words):
        score       = rescale_score_by_abs(scores[idx], max_s, min_s)
        output_text = output_text + span_word(w, score, colormap) + " "
    
    return output_text + "\n"

In [0]:
def rescale_score_by_abs (score, max_score, min_score):
    """
    Normalize the relevance value (=score), accordingly to the extremal relevance values (max_score and min_score), 
    for visualization with a diverging colormap.
    i.e. rescale positive relevance to the range [0.5, 1.0], and negative relevance to the range [0.0, 0.5],
    using the highest absolute relevance for linear interpolation.
    """
    
    # CASE 1: positive AND negative scores occur --------------------
    if max_score>0 and min_score<0:
    
        if max_score >= abs(min_score):   # deepest color is positive
            if score>=0:
                return 0.5 + 0.5*(score/max_score)
            else:
                return 0.5 - 0.5*(abs(score)/max_score)

        else:                             # deepest color is negative
            if score>=0:
                return 0.5 + 0.5*(score/abs(min_score))
            else:
                return 0.5 - 0.5*(score/min_score)   
    
    # CASE 2: ONLY positive scores occur -----------------------------       
    elif max_score>0 and min_score>=0: 
        if max_score == min_score:
            return 1.0
        else:
            return 0.5 + 0.5*(score/max_score)
    
    # CASE 3: ONLY negative scores occur -----------------------------
    elif max_score<=0 and min_score<0: 
        if max_score == min_score:
            return 0.0
        else:
            return 0.5 - 0.5*(score/min_score)
          

def getRGB (c_tuple):
    return "#%02x%02x%02x"%(int(c_tuple[0]*255), int(c_tuple[1]*255), int(c_tuple[2]*255))

     
def span_word (word, score, colormap):
    return "<span style=\"background-color:"+getRGB(colormap(score))+"\">"+word+"</span>"

In [0]:
len(words)

In [0]:
len(R_words_GI)

In [0]:
from IPython.display import display, HTML

print ("prediction scores:       ",   s)
print ("\nSA/GI target class:      ", target_class)
print ("\nSA relevances:")

# for idx, w in enumerate(words):
#     print ("\t\t\t" + "{:8.5f}".format(R_words_SA[idx]) + "\t" + w)
    
#print ("\nSA heatmap:")    
#display(HTML(html_heatmap(words, R_words_SA)))
print ("\nGI relevances:")
for idx, w in enumerate(words):
    print ("\t\t\t" + "{:8.2f}".format(R_words_GI[idx]) + "\t" + w)
print ("\nGI heatmap:")    
display(HTML(html_heatmap(words, R_words_GI[:100])))